# AWS Inferentia inference on Amazon EC2 Inf1 instance
This example demonstrates AWS Inferentia inference with TensorFlow and AWS Neuron SDK compiler and runtime

This example was tested on Amazon EC2 `inf1.xlarge` the following AWS Deep Learning AMI: 
`Deep Learning AMI (Ubuntu 18.04) Version 35.0`

Run this notebook using the following conda environment:
`aws_neuron_tensorflow_p36`

Prepare your imagenet validation TFRecord files using the following helper script:
https://github.com/tensorflow/models/blob/archive/research/inception/inception/data/download_and_preprocess_imagenet.sh

Save it to `/home/ubuntu/datasets/` or update the dataset location in the `get_dataset()` function

In [1]:
# !pip install matplotlib pandas

In [2]:
!/opt/aws/neuron/bin/neuron-cli reset
import os
import time
import shutil
import json
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.neuron as tfn
import tensorflow.compat.v1.keras as keras
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from concurrent import futures
from itertools import compress

print('test')

test


In [3]:
# https://github.com/tensorflow/tensorflow/issues/29931
temp = tf.zeros([8, 224, 224, 3])
_ = tf.keras.applications.vgg16.preprocess_input(temp)

### Resnet50 FP32 saved model

In [4]:
# # Export SavedModel
model_type = 'vgg16'

saved_model_dir = f'{model_type}_saved_model'
shutil.rmtree(saved_model_dir, ignore_errors=True)

keras.backend.set_learning_phase(0)
model = VGG16(weights='imagenet')
# tf.saved_model.simple_save(session = keras.backend.get_session(),
#                            export_dir = saved_model_dir,
#                            inputs = {'input_1:0': model.inputs[0]},
#                            outputs = {'probs/Softmax:0': model.outputs[0]})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Compile models with different batch sizes and cores

In [5]:
def compile_inf1_model(saved_model_dir, inf1_model_dir, batch_size=1, num_cores=1, use_static_weights=False):
    print(f'-----------batch size: {batch_size}, num cores: {num_cores}----------')
    print('Compiling...')
    
    compiled_model_dir = f'{model_type}_batch_{batch_size}_inf1_cores_{num_cores}'
    inf1_compiled_model_dir = os.path.join(inf1_model_dir, compiled_model_dir)
    shutil.rmtree(inf1_compiled_model_dir, ignore_errors=True)

    example_input = np.zeros([batch_size,224,224,3], dtype='float32')

    compiler_args = ['--verbose','1', '--neuroncore-pipeline-cores', str(num_cores)]
    if use_static_weights:
        compiler_args.append('--static-weights')
    
    start_time = time.time()
    compiled_res = tfn.saved_model.compile(model_dir = saved_model_dir,
                            model_feed_dict={'input_1:0': example_input},
                            new_model_dir = inf1_compiled_model_dir,
                            dynamic_batch_size=True,
                            compiler_args = compiler_args)
    print(f'Compile time: {time.time() - start_time}')
    
    compile_success = False
    perc_on_inf = compiled_res['OnNeuronRatio'] * 100
    if perc_on_inf > 50:
        compile_success = True
            
    print(inf1_compiled_model_dir)
    print(compiled_res)
    print('----------- Done! ----------- \n')
    
    return compile_success

### Use `tf.data` to read ImageNet validation dataset

In [6]:
def deserialize_image_record(record):
    feature_map = {'image/encoded': tf.io.FixedLenFeature([], tf.string, ''),
                  'image/class/label': tf.io.FixedLenFeature([1], tf.int64, -1),
                  'image/class/text': tf.io.FixedLenFeature([], tf.string, '')}
    obj = tf.io.parse_single_example(serialized=record, features=feature_map)
    imgdata = obj['image/encoded']
    label = tf.cast(obj['image/class/label'], tf.int32)   
    label_text = tf.cast(obj['image/class/text'], tf.string)   
    return imgdata, label, label_text

def val_preprocessing(record):
    imgdata, label, label_text = deserialize_image_record(record)
    label -= 1
    image = tf.io.decode_jpeg(imgdata, channels=3, 
                              fancy_upscaling=False, 
                              dct_method='INTEGER_FAST')

    shape = tf.shape(image)
    height = tf.cast(shape[0], tf.float32)
    width = tf.cast(shape[1], tf.float32)
    side = tf.cast(tf.convert_to_tensor(256, dtype=tf.int32), tf.float32)

    scale = tf.cond(tf.greater(height, width),
                  lambda: side / width,
                  lambda: side / height)
    
    new_height = tf.cast(tf.math.rint(height * scale), tf.int32)
    new_width = tf.cast(tf.math.rint(width * scale), tf.int32)
    
    image = tf.image.resize(image, [new_height, new_width], method='bicubic')
    image = tf.image.resize_with_crop_or_pad(image, 224, 224)
    
    image = tf.keras.applications.vgg16.preprocess_input(image)
    
    return image, label, label_text

def get_dataset(batch_size, use_cache=False):
    data_dir = '/home/ubuntu/datasets/*'
    files = tf.io.gfile.glob(os.path.join(data_dir))
    dataset = tf.data.TFRecordDataset(files)
    
    dataset = dataset.map(map_func=val_preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat(count=1)
    
    if use_cache:
        shutil.rmtree('tfdatacache', ignore_errors=True)
        os.mkdir('tfdatacache')
        dataset = dataset.cache(f'./tfdatacache/imagenet_val')
    
    return dataset

## Single AWS Inferentia chip execution
* Single core compiled models with automatic data parallel model upto 4 cores
* Multi-core compiled models for pipeline execution

In [7]:
def inf1_predict_benchmark_single_threaded(neuron_saved_model_name, batch_size, user_batch_size, num_cores, use_cache=False, warm_up=10):
    print(f'Running model {neuron_saved_model_name}, user_batch_size: {user_batch_size}\n')

    model_inf1 = tf.contrib.predictor.from_saved_model(neuron_saved_model_name)

    iter_times = []
    pred_labels = []
    actual_labels = []
    display_threshold = 0
    warm_up = 10

    ds = get_dataset(user_batch_size, use_cache)

    ds_iter = ds.make_initializable_iterator()
    ds_next = ds_iter.get_next()
    ds_init_op = ds_iter.initializer

    with tf.Session() as sess:
        if use_cache:
            sess.run(ds_init_op)
            print('\nCaching dataset ...')
            start_time = time.time()
            try:
                while True:
                    (validation_ds,label,_) = sess.run(ds_next)
            except tf.errors.OutOfRangeError:
                pass
            print(f'Caching finished: {time.time()-start_time} sec')  

        try:
            sess.run(ds_init_op)
            counter = 0
            
            total_datas = 1000
            display_every = 100
            display_threshold = display_every
            
            ipname = list(model_inf1.feed_tensors.keys())[0]
            resname = list(model_inf1.fetch_tensors.keys())[0]
            
            walltime_start = time.time()
            warmup_time = []
            extend_time = []
            while True:
                sess_start = time.time()
                (validation_ds,batch_labels,_) = sess.run(ds_next)
                
                model_feed_dict={ipname: validation_ds}
                warmup_start = time.time()
                if counter == 0:
                    for i in range(warm_up):
                        _ = model_inf1(model_feed_dict);                    
                warmup_time.append(time.time() - warmup_start)
                start_time =time.time()
                inf1_results = model_inf1(model_feed_dict);
                iter_times.append(time.time() - start_time)
                
                extend_start = time.time()
                actual_labels.extend(label for label_list in batch_labels for label in label_list)
                pred_labels.extend(list(np.argmax(inf1_results[resname], axis=1)))
                extend_time.append(time.time() - extend_start)
                
                if counter*user_batch_size >= display_threshold:
                    print(f'Images {counter*user_batch_size}/{total_datas}. Average i/s {np.mean(user_batch_size/np.array(iter_times[-display_every:]))}')
                    display_threshold+=display_every

                counter+=1
        except tf.errors.OutOfRangeError:
            pass
    
    labeling_start = time.time()
    acc_inf1 = np.sum(np.array(actual_labels) == np.array(pred_labels))/len(actual_labels)
    iter_times = np.array(iter_times)
    labeling_time = time.time() - labeling_start
    
    results = pd.DataFrame(columns = [f'inf1_compiled_batch_size_{batch_size}_compiled_cores_{num_cores}'])
    results.loc['compiled_batch_size']     = [batch_size]
    results.loc['user_batch_size']         = [user_batch_size]
    results.loc['accuracy']                = [acc_inf1]
    results.loc['prediction_time']         = [np.sum(iter_times)]
    results.loc['warmup_time']             = [np.sum(np.array(warmup_time))]
    results.loc['extend_time']             = [np.sum(np.array(extend_time))]
    results.loc['labeling_time']           = [np.sum(np.array(labeling_time))]
    results.loc['wall_time']               = [time.time() - walltime_start]
    results.loc['images_per_sec_mean']     = [np.mean(user_batch_size / iter_times)]
    results.loc['images_per_sec_std']      = [np.std(user_batch_size / iter_times, ddof=1)]
    results.loc['latency_mean']            = [np.mean(iter_times) * 1000]
    results.loc['latency_99th_percentile'] = [np.percentile(iter_times, q=99, interpolation="lower") * 1000]
    results.loc['latency_median']          = [np.median(iter_times) * 1000]
    results.loc['latency_min']             = [np.min(iter_times) * 1000]
    display(results.T)
#     shutil.rmtree(neuron_saved_model_name, ignore_errors=True)
    return results, iter_times

In [9]:
inf1_model_dir = f'{model_type}_inf1_saved_models'
saved_model_dir = f'{model_type}_saved_model'


# testing batch size
batch_list = [16]
num_of_cores = [1,2,3,4]

inf1_model_dir = f'{model_type}_inf1_saved_models'

for batch_size in batch_list:
    iter_ds = pd.DataFrame()
    results = pd.DataFrame()
    for num_cores in num_of_cores:
        opt ={'batch_size': batch_size, 'num_cores': num_of_cores}
        compiled_model_dir = f'{model_type}_batch_{batch_size}_inf1_cores_{num_cores}'
        inf1_compiled_model_dir = os.path.join(inf1_model_dir, compiled_model_dir)

        print(f'inf1_compiled_model_dir: {inf1_compiled_model_dir}')
        col_name = lambda opt: f'inf1_{batch_size}_multicores_{num_cores}'

        res, iter_times = inf1_predict_benchmark_single_threaded(inf1_compiled_model_dir,
                                                                         batch_size = batch_size,
                                                                         user_batch_size = batch_size*10,
                                                                         num_cores = num_cores,
                                                                         use_cache=False, 
                                                                         warm_up=10)

        iter_ds = pd.concat([iter_ds, pd.DataFrame(iter_times, columns=[col_name(opt)])], axis=1)
        results = pd.concat([results, res], axis=1)

    display(results)

inf1_compiled_model_dir: vgg16_inf1_saved_models/vgg16_batch_16_inf1_cores_1
Running model vgg16_inf1_saved_models/vgg16_batch_16_inf1_cores_1, user_batch_size: 160

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Images 160/1000. Average i/s 704.4305637826494
Images 320/1000. Average i/s 712.6685727191417
Images 480/1000. Average i/s 708.8029235032435
Images 640/1000. Average i/s 711.0405707121217
Images 800/1000. Average i/s 710.8271270303061
Images 960/1000. Average i/s 881.0125045554158


,compiled_batch_size,user_batch_size,accuracy,prediction_time,warmup_time,extend_time,labeling_time,wall_time,images_per_sec_mean,images_per_sec_std,latency_mean,latency_99th_percentile,latency_median,latency_min
inf1_compiled_batch_size_16_compiled_cores_1,16,160,0.845,1.43533,15.228,0.00202918,0.000211,16.931,881.013,450.543,205.047,229.487,222.225,84.1165


inf1_compiled_model_dir: vgg16_inf1_saved_models/vgg16_batch_16_inf1_cores_2
Running model vgg16_inf1_saved_models/vgg16_batch_16_inf1_cores_2, user_batch_size: 160

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Images 160/1000. Average i/s 266.13583419025406
Images 320/1000. Average i/s 265.90729046579344
Images 480/1000. Average i/s 265.774202822645
Images 640/1000. Average i/s 266.1460660428953
Images 800/1000. Average i/s 266.25162107241687
Images 960/1000. Average i/s 350.95369195105974


,compiled_batch_size,user_batch_size,accuracy,prediction_time,warmup_time,extend_time,labeling_time,wall_time,images_per_sec_mean,images_per_sec_std,latency_mean,latency_99th_percentile,latency_median,latency_min
inf1_compiled_batch_size_16_compiled_cores_2,16,160,0.846,3.79191,9.56145,0.00205636,0.000208378,13.6148,350.954,224.104,541.701,602.921,599.746,186.227


inf1_compiled_model_dir: vgg16_inf1_saved_models/vgg16_batch_16_inf1_cores_3
Running model vgg16_inf1_saved_models/vgg16_batch_16_inf1_cores_3, user_batch_size: 160

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Images 160/1000. Average i/s 269.7990099315389
Images 320/1000. Average i/s 270.01029768818125
Images 480/1000. Average i/s 269.52924481924794
Images 640/1000. Average i/s 269.8826399323409
Images 800/1000. Average i/s 270.08212762488716
Images 960/1000. Average i/s 355.638432029497


,compiled_batch_size,user_batch_size,accuracy,prediction_time,warmup_time,extend_time,labeling_time,wall_time,images_per_sec_mean,images_per_sec_std,latency_mean,latency_99th_percentile,latency_median,latency_min
inf1_compiled_batch_size_16_compiled_cores_3,16,160,0.846,3.73866,9.61226,0.00206017,0.00196338,13.6142,355.638,226.364,534.095,594.758,591.32,184.125


inf1_compiled_model_dir: vgg16_inf1_saved_models/vgg16_batch_16_inf1_cores_4
Running model vgg16_inf1_saved_models/vgg16_batch_16_inf1_cores_4, user_batch_size: 160

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Images 160/1000. Average i/s 398.6086335602753
Images 320/1000. Average i/s 397.86416720222195
Images 480/1000. Average i/s 397.86806069397755
Images 640/1000. Average i/s 398.5991400751215
Images 800/1000. Average i/s 399.04987006285637
Images 960/1000. Average i/s 499.91757485770984


,compiled_batch_size,user_batch_size,accuracy,prediction_time,warmup_time,extend_time,labeling_time,wall_time,images_per_sec_mean,images_per_sec_std,latency_mean,latency_99th_percentile,latency_median,latency_min
inf1_compiled_batch_size_16_compiled_cores_4,16,160,0.846,2.55056,7.79514,0.00204444,0.000212193,10.6176,499.918,266.879,364.366,403.24,399.569,144.78


,inf1_compiled_batch_size_16_compiled_cores_1,inf1_compiled_batch_size_16_compiled_cores_2,inf1_compiled_batch_size_16_compiled_cores_3,inf1_compiled_batch_size_16_compiled_cores_4
compiled_batch_size,16,16,16,16
user_batch_size,160,160,160,160
accuracy,0.845,0.846,0.846,0.846
prediction_time,1.43533,3.79191,3.73866,2.55056
warmup_time,15.228,9.56145,9.61226,7.79514
extend_time,0.00202918,0.00205636,0.00206017,0.00204444
labeling_time,0.000211,0.000208378,0.00196338,0.000212193
wall_time,16.931,13.6148,13.6142,10.6176
images_per_sec_mean,881.013,350.954,355.638,499.918
images_per_sec_std,450.543,224.104,226.364,266.879
